In [9]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

In [2]:


def hngarch_sim(model=None, n=1000, innov=None, n_start=100, start_innov=None, rand_gen=np.random.normal):
    if model is None:
        model = {'lambda': 4, 'omega': 4*0.0002, 'alpha': 0.3*0.0002, 'beta': 0.3, 'gamma': 0, 'rf': 0}
    
    if innov is None:
        innov = rand_gen(n)
    if start_innov is None:
        start_innov = rand_gen(n_start)
    
    lambda_ = model['lambda']
    omega = model['omega']
    alpha = model['alpha']
    beta = model['beta']
    gamma = model['gamma']
    rfr = model['rf']
    
    x = h = Z = np.concatenate([start_innov, innov])
    nt = n_start + n
    
    h[0] = (omega + alpha) / (1 - alpha*gamma*gamma - beta)
    x[0] = rfr + lambda_*h[0] + np.sqrt(h[0]) * Z[0]
    
    for i in range(1, nt):
        h[i] = omega + alpha*(Z[i-1] - gamma*np.sqrt(h[i-1]))**2 + beta*h[i-1]
        x[i] = rfr + lambda_*h[i] + np.sqrt(h[i]) * Z[i]
    
    return x[n_start:]

def hngarch_fit(x, model=None, symmetric=True, trace=False, title=None, description=None):
    if model is None:
        model = {'lambda': -0.5, 'omega': np.var(x), 'alpha': 0.1*np.var(x),
                 'beta': 0.1, 'gamma': 0, 'rf': 0}
    
    rfr = model['rf']
    lambda_ = model['lambda']
    omega = model['omega']
    alpha = model['alpha']
    beta = model['beta']
    gam = model['gamma']
    
    params = {'lambda': lambda_, 'omega': omega, 'alpha': alpha,
              'beta': beta, 'gamma': gam, 'rf': rfr}
    
    par_omega = -np.log((1-omega)/omega)
    par_alpha = -np.log((1-alpha)/alpha)
    par_beta = -np.log((1-beta)/beta)
    par_start = [lambda_, par_omega, par_alpha, par_beta]
    if not symmetric:
        par_start.append(gam)
    
    def llh_hngarch(par, trace, symmetric, rfr, x):
        h = Z = x.copy()
        lambda_ = par[0]
        omega = 1 / (1+np.exp(-par[1]))
        alpha = 1 / (1+np.exp(-par[2]))
        beta = 1 / (1+np.exp(-par[3]))
        gam = par[4] if not symmetric else 0
        
        h[0] = (omega + alpha) / (1 - alpha*gam*gam - beta)
        Z[0] = (x[0] - rfr - lambda_*h[0]) / np.sqrt(h[0])
        for i in range(1, len(Z)):
            h[i] = omega + alpha * (Z[i-1] - gam * np.sqrt(h[i-1]))**2 + beta * h[i-1]
            Z[i] = (x[i] - rfr - lambda_*h[i]) / np.sqrt(h[i])
        
        llh_hngarch = -np.sum(np.log(norm.pdf(Z) / np.sqrt(h)))
        if trace:
            print("Parameter Estimate")
            print([lambda_, omega, alpha, beta, gam])
        
        return llh_hngarch, Z, h
    
    result = minimize(lambda par: llh_hngarch(par, trace, symmetric, rfr, x)[0], 
                      par_start, method='Nelder-Mead')
    
    opt = {}
    opt['minimum'] = -result.fun + len(x)*np.sqrt(2*np.pi)
    opt['estimate'] = result.x
    opt['params'] = params
    opt['symmetric'] = symmetric
    
    final, opt['z'], opt['h'] = llh_hngarch(result.x, False, symmetric, rfr, x)
    
    lambda_ = opt['estimate'][0]
    omega = opt['estimate'][1] = 1 / (1+np.exp(-opt['estimate'][1]))
    alpha = opt['estimate'][2] = 1 / (1+np.exp(-opt['estimate'][2]))
    beta = opt['estimate'][3] = 1 / (1+np.exp(-opt['estimate'][3]))
    if symmetric:
        opt['estimate'] = np.append(opt['estimate'], 0)
    gam = opt['estimate'][4]
    
    opt['model'] = {'lambda': lambda_, 'omega': omega, 'alpha': alpha,
                    'beta': beta, 'gamma': gam, 'rf': rfr}
    opt['x'] = x
    opt['persistence'] = beta + alpha*gam*gam
    opt['sigma2'] = (omega + alpha) / (1 - opt['persistence'])
    
    if trace:
        print(opt['estimate'])
    
    opt['title'] = title if title else "Heston-Nandi Garch Parameter Estimation"
    opt['description'] = description if description else "No description provided"
    
    return opt

# The print and summary methods would need to be implemented differently in Python,
# as Python doesn't have the same class system as R. You might want to create a custom
# class for the hngarch results and implement __str__ and __repr__ methods.

def hngarch_stats(model):
    lambda_ = model['lambda']
    omega = model['omega']
    alpha = model['alpha']
    beta = model['beta']
    gamma = model['gamma']
    
    expect2, expect4, expect6, expect8 = 1, 3, 15, 105
    
    if gamma == 0:
        persistence = beta
        meansigma2 = (omega+alpha) / (1-beta)
        meansigma4 = (omega**2 + 2*omega*alpha + 2*omega*beta*meansigma2 +
                      3*alpha**2 + 2*alpha*beta*meansigma2) / (1 - beta**2)
        meansigma6 = (omega**3 + 3*omega**2*alpha + 3*omega**2*beta*meansigma2 +
                      9*omega*alpha**2 + 6*omega*alpha*beta*meansigma2 +
                      3*omega*beta**2*meansigma4 + 15*alpha**3 +
                      9*alpha**2*beta*meansigma2 + 3*alpha*beta**2*meansigma4) / (1-beta**3)
        meansigma8 = (omega**4 + expect8*alpha**4 + 12*omega**2*alpha*beta*meansigma2 +
                      60*alpha**3*beta*meansigma2 + 18*alpha**2*beta**2*meansigma4 +
                      4*alpha*beta**3*meansigma6 + 36*omega*alpha**2*beta*meansigma2 +
                      12*omega*alpha*beta**2*meansigma4 + 4*omega**3*alpha +
                      4*omega**3*beta*meansigma2 + 18*omega**2*alpha**2 +
                      6*omega**2*beta**2*meansigma4 + 60*omega*alpha**3 +
                      4*omega*beta**3*meansigma6) / (1 - beta**4)
    else:
        persistence = beta + alpha*gamma**2
        meansigma2 = (omega+alpha) / (1-beta-alpha*gamma**2)
        meansigma4 = (omega**2 + 2*omega*beta*meansigma2 +
                      alpha**2*expect4 + 2*beta*meansigma2*alpha*expect2 +
                      6*alpha**2*expect2*gamma**2*meansigma2 +
                      2*omega*alpha*gamma**2*meansigma2 + 2*omega*alpha*expect2) / \
                     (1 - beta**2 - 2*beta*alpha*gamma**2 - alpha**2*gamma**4)
        meansigma6 = (3*omega*alpha**2*expect4 + 3*omega**2*alpha*gamma**2*meansigma2 +
                      3*beta*meansigma2*alpha**2*expect4 +
                      3*beta**2*meansigma4*alpha*expect2 +
                      15*alpha**3*expect4*gamma**2*meansigma2 +
                      15*alpha**3*expect2*gamma**4*meansigma4 +
                      3*omega*alpha**2*gamma**4*meansigma4 +
                      3*omega**2*beta*meansigma2 + 3*omega**2*alpha*expect2 +
                      3*omega*beta**2*meansigma4 + omega**3 + alpha**3*expect6 +
                      18*beta*meansigma4*alpha**2*expect2*gamma**2 +
                      6*omega*beta*meansigma2*alpha*expect2 +
                      6*omega*beta*meansigma4*alpha*gamma**2 +
                      18*omega*alpha**2*expect2*gamma**2*meansigma2) / \
                     (1 - 3*beta**2*alpha*gamma**2 - 3*beta*alpha**2*gamma**4 -
                      alpha**3*gamma**6 - beta**3)
        meansigma8 = (omega**4 + alpha**4*expect8 +
                      6*omega**2*alpha**2*expect4 + 4*omega**3*beta*meansigma2 +
                      4*omega**3*alpha*expect2 + 6*omega**2*beta**2*meansigma4 +
                      4*omega*beta**3*meansigma6 + 4*omega*alpha**3*expect6 +
                      12*omega**2*beta*meansigma2*alpha*expect2 +
                      12*omega**2*beta*meansigma4*alpha*gamma**2 +
                      36*omega**2*alpha**2*expect2*gamma**2*meansigma2 +
                      4*omega**3*alpha*gamma**2*meansigma2 +
                      6*omega**2*alpha**2*gamma**4*meansigma4 +
                      6*beta**2*meansigma4*alpha**2*expect4 +
                      4*beta**3*meansigma6*alpha*expect2 +
                      4*beta*meansigma2*alpha**3*expect6 +
                      28*alpha**4*expect6*gamma**2*meansigma2 +
                      70*alpha**4*expect4*gamma**4*meansigma4 +
                      28*alpha**4*expect2*gamma**6*meansigma6 +
                      4*omega*alpha**3*gamma**6*meansigma6 +
                      60*beta*meansigma4*alpha**3*expect4*gamma**2 +
                      60*beta*meansigma6*alpha**3*expect2*gamma**4 +
                      36*beta**2*meansigma6*alpha**2*expect2*gamma**2 +
                      12*omega*beta*meansigma2*alpha**2*expect4 +
                      12*omega*beta**2*meansigma4*alpha*expect2 +
                      12*omega*beta**2*meansigma6*alpha*gamma**2 +
                      12*omega*beta*meansigma6*alpha**2*gamma**4 +
                      60*omega*alpha**3*expect4*gamma**2*meansigma2 +
                      60*omega*alpha**3*expect2*gamma**4*meansigma4 +
                      72*omega*beta*meansigma4*alpha**2*expect2*gamma**2) / \
                     (1 - beta**4 - alpha**4*gamma**8 - 4*beta**3*alpha*gamma**2 -
                      6*beta**2*alpha**2*gamma**4 - 4*beta*alpha**3*gamma**6)
    
    leverage = -2*alpha*gamma*meansigma2
    
    uc_mean = lambda_*meansigma2
    uc_variance = lambda_**2*(meansigma4 - meansigma2**2) + meansigma2
    uc_skewness = (3*lambda_*meansigma4 - 3*lambda_*meansigma2**2 +
                   lambda_**3*meansigma6 - 3*lambda_**3*meansigma2*meansigma4 +
                   2*lambda_**3*meansigma2**3) / uc_variance**1.5
    uc_kurtosis = (meansigma4*3 + 6*lambda_**2*meansigma6 -
                   12*lambda_**2*meansigma2*meansigma4 + 6*lambda_**2*meansigma2**3 +
                   lambda_**4*meansigma8 - 4*lambda_**4*meansigma2*meansigma6 +
                   6*lambda_**4*meansigma2**2*meansigma4 -
                   3*lambda_**4*meansigma2**4) / uc_variance**2
    
    return {
        'mean': uc_mean,
        'variance': uc_variance,
        'skewness': uc_skewness,
        'kurtosis': uc_kurtosis,
        'persistence': persistence,
        'leverage': leverage,
        'meansigma2': meansigma2,
        'meansigma4': meansigma4,
        'meansigma6': meansigma6,
        'meansigma8': meansigma8
    }

In [3]:
import numpy as np
from scipy import integrate
import cmath

In [4]:


def HNGOption(TypeFlag, model, S, X, Time_inDays, r_daily):
    """
    Calculates the price of a HN-GARCH option.
    
    The function calculates the price of a Heston-Nandi GARCH(1,1)
    call or put option.
    """
    
    # Option Type:
    TypeFlag = TypeFlag[0]
    
    # Integrate:
    call1 = integrate.quad(lambda phi: fstarHN(phi, 1, model, S, X, Time_inDays, r_daily), 0, np.inf)[0]
    call2 = integrate.quad(lambda phi: fstarHN(phi, 0, model, S, X, Time_inDays, r_daily), 0, np.inf)[0]
    
    # Compute Call Price:
    call_price = S/2 + np.exp(-r_daily*Time_inDays) * call1 - \
        X * np.exp(-r_daily*Time_inDays) * (1/2 + call2)
    
    # Select Option Price:
    price = None
    if TypeFlag == "c":
        price = call_price
    if TypeFlag == "p":
        price = call_price + X*np.exp(-r_daily*Time_inDays) - S
    
    # Return Value:
    return {"price": price}

def fstarHN(phi, const, model, S, X, Time_inDays, r_daily):
    """Internal Function"""
    
    # Model Parameters:
    lambda_ = -1/2
    omega = model['omega']
    alpha = model['alpha']
    gamma = model['gamma'] + model['lambda'] + 1/2
    beta = model['beta']
    sigma2 = (omega + alpha)/(1 - beta - alpha * gamma**2)
    
    # Function to be integrated:
    cphi0 = phi * 1j
    cphi = cphi0 + const
    a = cphi * r_daily
    b = lambda_*cphi + cphi*cphi/2
    for i in range(2, Time_inDays+1):
        a = a + cphi*r_daily + b*omega - np.log(1-2*alpha*b)/2
        b = cphi*(lambda_+gamma) - gamma**2/2 + beta*b + \
            0.5*(cphi-gamma)**2/(1-2*alpha*b)
    f = (np.exp(-cphi0*np.log(X)+cphi*np.log(S)+a+b*sigma2)/cphi0).real/np.pi
    
    return f

def HNGGreeks(Selection, TypeFlag, model, S, X, Time_inDays, r_daily):
    """
    Calculates the Greeks of a HN-GARCH option.
    
    The function calculates the delta and gamma Greeks of
    a Heston Nandi GARCH(1,1) call or put option.
    """
    
    # Type Flags:
    Selection = Selection[0]
    TypeFlag = TypeFlag[0]
    
    # Delta:
    if Selection == "Delta":
        delta1 = integrate.quad(lambda phi: fdeltaHN(phi, 1, model, S, X, Time_inDays, r_daily), 0, np.inf)[0]
        delta2 = integrate.quad(lambda phi: fdeltaHN(phi, 0, model, S, X, Time_inDays, r_daily), 0, np.inf)[0]
        
        # Compute Call and Put Delta:
        greek = 1/2 + np.exp(-r_daily*Time_inDays) * delta1 - \
            X * np.exp(-r_daily*Time_inDays) * delta2
        if TypeFlag == "p":
            greek = greek - 1
    
    # Gamma:
    if Selection == "Gamma":
        gamma1 = integrate.quad(lambda phi: fgammaHN(phi, 1, model, S, X, Time_inDays, r_daily), 0, np.inf)[0]
        gamma2 = integrate.quad(lambda phi: fgammaHN(phi, 0, model, S, X, Time_inDays, r_daily), 0, np.inf)[0]
        
        # Compute Call and Put Gamma:
        greek = np.exp(-r_daily*Time_inDays) * gamma1 - \
            X * np.exp(-r_daily*Time_inDays) * gamma2
    
    return greek

def fdeltaHN(phi, const, model, S, X, Time_inDays, r_daily):
    """Function to be integrated for Delta"""
    cphi0 = phi * 1j
    cphi = cphi0 + const
    fdelta = cphi * fHN(phi, const, model, S, X, Time_inDays, r_daily) / S
    return fdelta.real

def fgammaHN(phi, const, model, S, X, Time_inDays, r_daily):
    """Function to be integrated for Gamma"""
    cphi0 = phi * 1j
    cphi = cphi0 + const
    fgamma = cphi * (cphi - 1) * fHN(phi, const, model, S, X, Time_inDays, r_daily) / S**2
    return fgamma.real

def fHN(phi, const, model, S, X, Time_inDays, r_daily):
    """Internal Function"""
    
    # Model Parameters:
    lambda_ = -1/2
    omega = model['omega']
    alpha = model['alpha']
    gamma = model['gamma'] + model['lambda'] + 1/2
    beta = model['beta']
    sigma2 = (omega + alpha)/(1 - beta - alpha * gamma**2)
    
    # Function to be integrated:
    cphi0 = phi * 1j
    cphi = cphi0 + const
    a = cphi * r_daily
    b = lambda_*cphi + cphi*cphi/2
    for i in range(2, Time_inDays+1):
        a = a + cphi*r_daily + b*omega - cmath.log(1-2*alpha*b)/2
        b = cphi*(lambda_+gamma) - gamma**2/2 + beta*b + \
            0.5*(cphi-gamma)**2/(1-2*alpha*b)
    fun = cmath.exp(-cphi0*cmath.log(X)+cphi*cmath.log(S)+a+b*sigma2)/cphi0/np.pi
    
    return fun

def HNGCharacteristics(TypeFlag, model, S, X, Time_inDays, r_daily):
    """
    The function calculates the option price for the Heston
    Nandi Garch(1,1) option model together with the delta
    and gamma option sensitivities.
    """
    
    # Premium and Function Call to all Greeks
    TypeFlag = TypeFlag[0]
    premium = HNGOption(TypeFlag, model, S, X, Time_inDays, r_daily)
    delta = HNGGreeks(["Delta"], TypeFlag, model, S, X, Time_inDays, r_daily)
    gamma = HNGGreeks(["Gamma"], TypeFlag, model, S, X, Time_inDays, r_daily)
    
    # Return Value:
    return {"premium": premium, "delta": delta, "gamma": gamma}

In [5]:
import numpy as np
from scipy import integrate

################################################################################
# FUNCTION:           DESCRIPTION:
#  HNGOption           Computes Option Price from the HN-GARCH Formula
#  HNGGreeks           Calculates one of the Greeks of the HN-GARCH Formula
#  HNGCharacteristics  Computes Option Price and all Greeks of HN-GARCH Model
################################################################################

def HNGOption(TypeFlag, model, S, X, Time_inDays, r_daily):
    # Description:
    #   Calculates the price of a HN-GARCH option.

    # Details:
    #   The function calculates the price of a Heston-Nandi GARCH(1,1)
    #   call or put option.

    # FUNCTION:

    # Option Type:
    TypeFlag = TypeFlag[0]

    # Integrate:
    call1 = integrate.quad(fstarHN, 0, np.inf, args=(1, model, S, X, Time_inDays, r_daily))
    call2 = integrate.quad(fstarHN, 0, np.inf, args=(0, model, S, X, Time_inDays, r_daily))

    # Compute Call Price:
    call_price = S/2 + np.exp(-r_daily*Time_inDays) * call1[0] - \
        X * np.exp(-r_daily*Time_inDays) * (1/2 + call2[0])

    # Select Option Price:
    price = None
    if TypeFlag == "c":
        price = call_price
    if TypeFlag == "p":
        price = call_price + X*np.exp(-r_daily*Time_inDays) - S

    # Return Value:
    option = {
        'price': price,
        'call': 'HNGOption'  # This is a simplified version of match.call()
    }
    return option

def fstarHN(phi, const, model, S, X, Time_inDays, r_daily):
    # Internal Function:

    # Model Parameters:
    lambda_ = -1/2
    omega = model['omega']
    alpha = model['alpha']
    gamma = model['gamma'] + model['lambda'] + 1/2
    beta = model['beta']
    sigma2 = (omega + alpha)/(1 - beta - alpha * gamma**2)
    
    # Function to be integrated:
    cphi0 = phi * 1j
    cphi = cphi0 + const
    a = cphi * r_daily
    b = lambda_*cphi + cphi*cphi/2
    for i in range(2, Time_inDays + 1):
        a = a + cphi*r_daily + b*omega - np.log(1-2*alpha*b)/2
        b = cphi*(lambda_+gamma) - gamma**2/2 + beta*b + \
            0.5*(cphi-gamma)**2/(1-2*alpha*b)
    f = np.real(np.exp(-cphi0*np.log(X)+cphi*np.log(S)+a+b*sigma2)/cphi0)/np.pi

    # Return Value:
    return f

def HNGGreeks(Selection, TypeFlag, model, S, X, Time_inDays, r_daily):
    # Description:
    #   Calculates the Greeks of a HN-GARCH option.

    # Details:
    #   The function calculates the delta and gamma Greeks of
    #   a Heston Nandi GARCH(1,1) call or put option.

    # FUNCTION:

    # Type Flags:
    Selection = Selection[0]
    TypeFlag = TypeFlag[0]

    # Delta:
    if Selection == "Delta":
        # Integrate:
        delta1 = integrate.quad(fdeltaHN, 0, np.inf, args=(1, model, S, X, Time_inDays, r_daily))
        delta2 = integrate.quad(fdeltaHN, 0, np.inf, args=(0, model, S, X, Time_inDays, r_daily))
        
        # Compute Call and Put Delta:
        greek = 1/2 + np.exp(-r_daily*Time_inDays) * delta1[0] - \
            X * np.exp(-r_daily*Time_inDays) * delta2[0]
        if TypeFlag == "p":
            greek = greek - 1

    # Gamma:
    if Selection == "Gamma":
        # Integrate:
        gamma1 = integrate.quad(fgammaHN, 0, np.inf, args=(1, model, S, X, Time_inDays, r_daily))
        gamma2 = integrate.quad(fgammaHN, 0, np.inf, args=(0, model, S, X, Time_inDays, r_daily))
        
        # Compute Call and Put Gamma:
        greek = np.exp(-r_daily*Time_inDays) * gamma1[0] - \
            X * np.exp(-r_daily*Time_inDays) * gamma2[0]

    # Return Value:
    return greek

def fdeltaHN(phi, const, model, S, X, Time_inDays, r_daily):
    # Function to be integrated:
    cphi0 = phi * 1j
    cphi = cphi0 + const
    fdelta = cphi * \
        fHN(phi, const, model, S, X, Time_inDays, r_daily) / S

    # Return Value:
    return np.real(fdelta)

def fgammaHN(phi, const, model, S, X, Time_inDays, r_daily):
    # Function to be integrated:
    cphi0 = phi * 1j
    cphi = cphi0 + const
    fgamma = cphi * (cphi - 1) * \
        fHN(phi, const, model, S, X, Time_inDays, r_daily) / S**2

    # Return Value:
    return np.real(fgamma)

def fHN(phi, const, model, S, X, Time_inDays, r_daily):
    # Internal Function:

    # Model Parameters:
    lambda_ = -1/2
    omega = model['omega']
    alpha = model['alpha']
    gamma = model['gamma'] + model['lambda'] + 1/2
    beta = model['beta']
    sigma2 = (omega + alpha)/(1 - beta - alpha * gamma**2)
    
    # Function to be integrated:
    cphi0 = phi * 1j
    cphi = cphi0 + const
    a = cphi * r_daily
    b = lambda_*cphi + cphi*cphi/2
    for i in range(2, Time_inDays + 1):
        a = a + cphi*r_daily + b*omega - np.log(1-2*alpha*b)/2
        b = cphi*(lambda_+gamma) - gamma**2/2 + beta*b + \
            0.5*(cphi-gamma)**2/(1-2*alpha*b)
    fun = np.exp(-cphi0*np.log(X)+cphi*np.log(S)+a+b*sigma2)/cphi0/np.pi

    # Return Value:
    return fun

def HNGCharacteristics(TypeFlag, model, S, X, Time_inDays, r_daily):
    # Description:
    #   The function calculates the option price for the Heston
    #   Nandi Garch(1,1) option model together with the delta
    #   and gamma option sensitivies.

    # FUNCTION:

    # Premium and Function Call to all Greeks
    TypeFlag = TypeFlag[0]
    premium = HNGOption([TypeFlag], model, S, X, Time_inDays, r_daily)
    delta = HNGGreeks(["Delta"], [TypeFlag], model, S, X, Time_inDays, r_daily)
    gamma = HNGGreeks(["Gamma"], [TypeFlag], model, S, X, Time_inDays, r_daily)

    # Return Value:
    return {'premium': premium, 'delta': delta, 'gamma': gamma}

In [30]:
import numpy as np
from scipy import integrate
import cmath

import pandas_datareader.data as web  #Pandas Datareader
import mplfinance as mpf
from matplotlib.widgets import RangeSlider
import matplotlib.dates as dates
from mpl_toolkits.mplot3d import Axes3D
import pandas_datareader.data as web


from datetime import datetime

def HNGOption(TypeFlag, model, S, X, Time_inDays, r_daily):
    # Function to calculate the price of a HN-GARCH option

    def fstarHN(phi, const, model, S, X, Time_inDays, r_daily):
        # Internal function

        # Model Parameters
        lambda_ = -1/2
        omega = model['omega']
        alpha = model['alpha']
        gamma = model['gamma'] + model['lambda'] + 1/2
        beta = model['beta']
        sigma2 = (omega + alpha) / (1 - beta - alpha * gamma**2)

        # Function to be integrated
        cphi0 = phi * 1j
        cphi = cphi0 + const
        a = cphi * r_daily
        b = lambda_ * cphi + cphi * cphi / 2
        for i in range(2, Time_inDays + 1):
            a = a + cphi * r_daily + b * omega - np.log(1 - 2 * alpha * b) / 2
            b = cphi * (lambda_ + gamma) - gamma**2 / 2 + beta * b + 0.5 * (cphi - gamma)**2 / (1 - 2 * alpha * b)
        f = (np.exp(-cphi0 * np.log(X) + cphi * np.log(S) + a + b * sigma2) / cphi0 / np.pi).real

        return f

    # Integrate
    call1 = integrate.quad(fstarHN, 0, np.inf, args=(1, model, S, X, Time_inDays, r_daily))
    call2 = integrate.quad(fstarHN, 0, np.inf, args=(0, model, S, X, Time_inDays, r_daily))

    # Compute Call Price
    call_price = S / 2 + np.exp(-r_daily * Time_inDays) * call1[0] - X * np.exp(-r_daily * Time_inDays) * (1/2 + call2[0])

    # Select Option Price
    if TypeFlag == "c":
        price = call_price
    elif TypeFlag == "p":
        price = call_price + X * np.exp(-r_daily * Time_inDays) - S

    return {'price': price}

def HNGGreeks(Selection, TypeFlag, model, S, X, Time_inDays, r_daily):
    def fHN(phi, const, model, S, X, Time_inDays, r_daily):
        # Internal function

        # Model Parameters
        lambda_ = -1/2
        omega = model['omega']
        alpha = model['alpha']
        gamma = model['gamma'] + model['lambda'] + 1/2
        beta = model['beta']
        sigma2 = (omega + alpha) / (1 - beta - alpha * gamma**2)

        # Function to be integrated
        cphi0 = phi * 1j
        cphi = cphi0 + const
        a = cphi * r_daily
        b = lambda_ * cphi + cphi * cphi / 2
        for i in range(2, Time_inDays + 1):
            a = a + cphi * r_daily + b * omega - np.log(1 - 2 * alpha * b) / 2
            b = cphi * (lambda_ + gamma) - gamma**2 / 2 + beta * b + 0.5 * (cphi - gamma)**2 / (1 - 2 * alpha * b)
        fun = np.exp(-cphi0 * np.log(X) + cphi * np.log(S) + a + b * sigma2) / cphi0 / np.pi

        return fun

    def fdeltaHN(phi, const, model, S, X, Time_inDays, r_daily):
        cphi0 = phi * 1j
        cphi = cphi0 + const
        fdelta = cphi * fHN(phi, const, model, S, X, Time_inDays, r_daily) / S
        return fdelta.real

    def fgammaHN(phi, const, model, S, X, Time_inDays, r_daily):
        cphi0 = phi * 1j
        cphi = cphi0 + const
        fgamma = cphi * (cphi - 1) * fHN(phi, const, model, S, X, Time_inDays, r_daily) / S**2
        return fgamma.real

    if Selection == "Delta":
        delta1 = integrate.quad(fdeltaHN, 0, np.inf, args=(1, model, S, X, Time_inDays, r_daily))
        delta2 = integrate.quad(fdeltaHN, 0, np.inf, args=(0, model, S, X, Time_inDays, r_daily))
        greek = 1/2 + np.exp(-r_daily * Time_inDays) * delta1[0] - X * np.exp(-r_daily * Time_inDays) * delta2[0]
        if TypeFlag == "p":
            greek = greek - 1

    elif Selection == "Gamma":
        gamma1 = integrate.quad(fgammaHN, 0, np.inf, args=(1, model, S, X, Time_inDays, r_daily))
        gamma2 = integrate.quad(fgammaHN, 0, np.inf, args=(0, model, S, X, Time_inDays, r_daily))
        greek = np.exp(-r_daily * Time_inDays) * gamma1[0] - X * np.exp(-r_daily * Time_inDays) * gamma2[0]

    return greek

def HNGCharacteristics(TypeFlag, model, S, X, Time_inDays, r_daily):
    premium = HNGOption(TypeFlag, model, S, X, Time_inDays, r_daily)
    delta = HNGGreeks("Delta", TypeFlag, model, S, X, Time_inDays, r_daily)
    gamma = HNGGreeks("Gamma", TypeFlag, model, S, X, Time_inDays, r_daily)

    return {'premium': premium, 'delta': delta, 'gamma': gamma}

# Example usage:
if __name__ == "__main__":
    model = {
        'lambda': -0.42,
        'omega': 4.8e-6,
        'alpha': 4.0e-6,
        'beta': 0.85,
        'gamma': 184.25
    }
    #ASSET
    series_id  = 'DCOILBRENTEU'
    #DATES
    start_date = datetime(1994, 8, 31)
    end_date = datetime(2024, 7, 1)
    data =  web.DataReader(series_id, 'fred', start_date, end_date)
    S = X = data['DCOILBRENTEU'].iloc[-1]
    Time_inDays = 252
    r_daily = 0.05 / Time_inDays
    sigma_daily = np.sqrt((model['omega'] + model['alpha']) / (1 - model['beta'] - model['alpha'] * model['gamma']**2))

    print("Model parameters:")
    print(f"S = {S}, X = {X}, r_daily = {r_daily}, sigma_daily = {sigma_daily}")

    print("\nHNG Option prices:")
    for TypeFlag in ['c', 'p']:
        price = HNGOption(TypeFlag, model, S, X, Time_inDays, r_daily)['price']
        print(f"{TypeFlag.upper()}: {price}")

    print("\nHNG Greeks:")
    for Selection in ['Delta', 'Gamma']:
        greek = HNGGreeks(Selection, 'c', model, S, X, Time_inDays, r_daily)
        print(f"{Selection}: {greek}")

    print("\nHNG Characteristics:")
    characteristics = HNGCharacteristics('c', model, S, X, Time_inDays, r_daily)
    print(f"Premium: {characteristics['premium']['price']}")
    print(f"Delta: {characteristics['delta']}")
    print(f"Gamma: {characteristics['gamma']}")

Model parameters:
S = 86.57, X = 86.57, r_daily = 0.0001984126984126984, sigma_daily = 0.024887351562823654

HNG Option prices:
C: 15.29656607751803
P: 11.074497356544839

HNG Greeks:
Delta: 0.661949659637361
Gamma: 0.010829178094428134

HNG Characteristics:
Premium: 15.29656607751803
Delta: 0.661949659637361
Gamma: 0.010829178094428134
